In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

data = pd.read_csv("../../data/data_export.csv")

data["cited_by_count"] = pd.to_numeric(data["cited_by_count"], errors="coerce")
data = data.dropna(subset=["cited_by_count", "author_h_index", "journal_h_index", "institution_h_index"])

In [4]:
data = data[data['publication_year']<2024]

In [11]:
for category in data['predicted_category'].unique():
    cat_data = data[data['predicted_category']==category]
    X = cat_data[["author_h_index", "journal_h_index", "institution_h_index"]]
    y = cat_data["cited_by_count"]
    X_with_constant = sm.add_constant(X)
    model = sm.OLS(y, X_with_constant).fit()
    print(model.summary())


                            OLS Regression Results                            
Dep. Variable:         cited_by_count   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     231.8
Date:                Sun, 29 Dec 2024   Prob (F-statistic):          2.88e-145
Time:                        18:03:06   Log-Likelihood:                -52207.
No. Observations:                9691   AIC:                         1.044e+05
Df Residuals:                    9687   BIC:                         1.044e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   2.8478    